In [1]:
import pandas as pd
import statistics as stat
import math as m 

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read the input CSV dataset file
df = pd.read_csv('Dataset/weatherHistory.csv')

# Remove duplicated rows
df.drop_duplicates(subset=['Formatted Date'], inplace=True)

# Sort the dataset by "Formatted Date"
df['Formatted Date'] = pd.to_datetime(df['Formatted Date'], utc=True)
df.sort_values(by='Formatted Date', ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

# Create the target variabile: it's the value of "Summary" of i-th row when considering the target column at (i-1)-th row
df['target'] = df['Summary'].shift(-1)
df = df.loc[~df['target'].isna()]

In [3]:
df.shape

(96428, 13)

In [4]:
df.head()

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary,target
0,2005-12-31 23:00:00+00:00,Partly Cloudy,rain,0.577778,-4.050000,0.89,17.1143,140.0,9.9820,0.0,1016.66,Mostly cloudy throughout the day.,Mostly Cloudy
1,2006-01-01 00:00:00+00:00,Mostly Cloudy,rain,1.161111,-3.238889,0.85,16.6152,139.0,9.9015,0.0,1016.15,Mostly cloudy throughout the day.,Mostly Cloudy
2,2006-01-01 01:00:00+00:00,Mostly Cloudy,rain,1.666667,-3.155556,0.82,20.2538,140.0,9.9015,0.0,1015.87,Mostly cloudy throughout the day.,Overcast
3,2006-01-01 02:00:00+00:00,Overcast,rain,1.711111,-2.194444,0.82,14.4900,140.0,9.9015,0.0,1015.56,Mostly cloudy throughout the day.,Mostly Cloudy
4,2006-01-01 03:00:00+00:00,Mostly Cloudy,rain,1.183333,-2.744444,0.86,13.9426,134.0,9.9015,0.0,1014.98,Mostly cloudy throughout the day.,Mostly Cloudy


### Categoria di feature (A) 

Questa categoria di feature (A) sono semplici da calcolare perché ragionano riga per riga, senza andare a includere righe precendenti o successive.

### Assignment 1

Feature “is_daily_summary_very_length”: a partire dalla colonna “Daily Summary”, creare un indicatore booleano che è:
- True, se il numero di caratteri è superiore a 30
- False, altrimenti

In [5]:
t = df[['Daily Summary']].copy()

In [6]:
# Soluzione Leon
t['is_daily_summary_very_length'] = [True if len(df['Daily Summary'][i])>30 else False for i in (df.index)]

In [7]:
# Solution Edo
t['edo'] = t['Daily Summary'].apply(lambda row: True if len(row) > 30 else False)

In [8]:
# Solution Edo 2
t['edo2']= t['Daily Summary'].str.len() > 30

In [9]:
t.head()

,Daily Summary,is_daily_summary_very_length,edo,edo2
0,Mostly cloudy throughout the day.,True,True,True
1,Mostly cloudy throughout the day.,True,True,True
2,Mostly cloudy throughout the day.,True,True,True
3,Mostly cloudy throughout the day.,True,True,True
4,Mostly cloudy throughout the day.,True,True,True


In [10]:
#Verification 
#df['is_daily_summary_very_length'].value_counts()

In [11]:
df.head(3)

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary,target
0,2005-12-31 23:00:00+00:00,Partly Cloudy,rain,0.577778,-4.050000,0.89,17.1143,140.0,9.9820,0.0,1016.66,Mostly cloudy throughout the day.,Mostly Cloudy
1,2006-01-01 00:00:00+00:00,Mostly Cloudy,rain,1.161111,-3.238889,0.85,16.6152,139.0,9.9015,0.0,1016.15,Mostly cloudy throughout the day.,Mostly Cloudy
2,2006-01-01 01:00:00+00:00,Mostly Cloudy,rain,1.666667,-3.155556,0.82,20.2538,140.0,9.9015,0.0,1015.87,Mostly cloudy throughout the day.,Overcast


#### Assignment 2

Feature “week_of_year”: a partire dalla colonna “Formatted Date”, creare una variabile categorica che specifica il numero della settimana all’interno dell’anno.

In [12]:
df['Formatted Date'] = pd.to_datetime(df['Formatted Date'], utc=True)
df['week_of_year'] =df['Formatted Date'].dt.weekofyear

In [13]:
df.sort_values(by='week_of_year', ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

In [14]:
#df['week_of_year'].value_counts()

#### Assignment 3

Feature “is_month_start”: a partire dalla colonna “Formatted Date”, creare una variabile booleana che è:
- True, se la data che si sta considerando è l’inizio del mese
- False, altrimenti

In [15]:
df['is_month_start'] =df['Formatted Date'].dt.is_month_start

In [16]:
df['is_month_start'].value_counts()

False    93260
True      3168
Name: is_month_start, dtype: int64

#### Assignment 4

Feature “is_greater_than_avg”: a partire dalle colonne “Formatted Date” e “Visibility (km)”, creare una variabile booleana che è:
- True, se il valore nella riga i-esima è superiore della media della colonna “Visibility (km)” nella stessa data
- False, altrimenti

In [17]:
# Solution Leon
df['is_greater_than_avg'] = [True if (df['Visibility (km)'][i] > df['Visibility (km)'].mean()) else False for i in (df['Formatted Date'].index)]

In [18]:
# Solution Edo
t = df[['Formatted Date', 'Visibility (km)']].copy()
t['Formatted Date'] = t['Formatted Date'].dt.date
avg_t = t.groupby(by='Formatted Date')['Visibility (km)'].mean().reset_index()
x=list(avg_t.to_dict()['Formatted Date'].values())
y=list(avg_t.to_dict()['Visibility (km)'].values())
avg_dict = {str(x[i]): y[i] for i in range(len(x))}
t['edo'] = t.apply(
    lambda row: row['Visibility (km)'] > avg_dict[str(row['Formatted Date'])],
    axis=1
)

In [19]:
t['edo'].value_counts()

False    52536
True     43892
Name: edo, dtype: int64

In [20]:
df['is_greater_than_avg'].value_counts()

False    50998
True     45430
Name: is_greater_than_avg, dtype: int64

In [21]:
df.head()

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary,target,week_of_year,is_month_start,is_greater_than_avg
0,2008-01-06 23:00:00+00:00,Foggy,snow,0.000000,0.000000,0.92,3.1556,219.0,2.9302,0.0,1021.59,Foggy throughout the day.,Foggy,1,False,False
1,2009-01-04 16:00:00+00:00,Partly Cloudy,snow,-7.222222,-7.222222,0.96,1.6100,180.0,6.1180,0.0,1020.00,Foggy starting overnight continuing until afte...,Partly Cloudy,1,False,False
2,2009-01-04 17:00:00+00:00,Partly Cloudy,snow,-7.222222,-12.766667,0.88,12.8800,220.0,6.1180,0.0,1019.80,Foggy starting overnight continuing until afte...,Mostly Cloudy,1,False,False
3,2009-01-04 18:00:00+00:00,Mostly Cloudy,snow,-6.177778,-12.261111,0.83,16.1161,213.0,6.2146,0.0,1020.12,Foggy starting overnight continuing until afte...,Overcast,1,False,False
4,2009-01-04 19:00:00+00:00,Overcast,snow,-6.111111,-11.416667,0.88,12.8800,220.0,4.0250,0.0,1020.20,Foggy starting overnight continuing until afte...,Overcast,1,False,False


##  Categoria di feature (B)

 
Questa categoria di feature (B) invece seguono la stessa logica che abbiamo visto oggi, quindi provo a valorizzare queste feature usando i valori presenti in 1 o più riga/he precedente/i o successiva/e:

#### Assignment 5

Feature “Temperature (C) t-1”: a partire dalle colonne “Formatted Date” e “Temperature (C)”, riportare nella riga i-esima il valore della colonna “Temperature (C)” che si trova nella riga (i-1)-esima.

In [22]:
# Solution Leon
df['Temperature (C) t-1'] = None # inutile
df['Temperature (C) t-1'] = df['Temperature (C)'].shift(1)
df=df.dropna().reset_index(drop=True)

In [23]:
# Solution Edo
t = df[['Formatted Date', 'Temperature (C)']].copy()
t['edo'] = t['Temperature (C)'].shift(1)

In [33]:
df.tail()

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary,target,week_of_year,is_month_start,is_greater_than_avg,Temperature (C) t-1,same summary in the last 3 days,Humidity avg last 2 days
95905,2009-12-28 01:00:00+00:00,Mostly Cloudy,rain,2.222222,-1.566667,0.75,14.4900,280.0,15.1340,0.0,1006.60,Partly cloudy throughout the day.,Mostly Cloudy,53,False,True,-0.505556,None,0.750
95906,2009-12-28 02:00:00+00:00,Mostly Cloudy,rain,2.222222,-1.566667,0.75,14.4900,260.0,15.1340,0.0,1006.60,Partly cloudy throughout the day.,Mostly Cloudy,53,False,True,2.222222,None,0.775
95907,2009-12-28 03:00:00+00:00,Mostly Cloudy,rain,1.683333,-2.294444,0.80,14.8603,239.0,13.9587,0.0,1006.81,Partly cloudy throughout the day.,Mostly Cloudy,53,False,True,2.222222,None,0.865
95908,2010-01-01 19:00:00+00:00,Foggy,rain,6.111111,3.466667,0.93,12.8800,340.0,3.0590,0.0,992.20,Foggy in the morning.,Foggy,53,True,False,1.683333,None,0.870
95909,2016-01-03 22:00:00+00:00,Mostly Cloudy,snow,-7.416667,-14.633333,0.81,20.3021,190.0,11.9784,0.0,1015.98,Mostly cloudy throughout the day.,Overcast,53,False,True,6.111111,None,NaN


#### Assignment 6

Feature “same summary in the last 3 hours”: a partire dalle colonne “Formatted Date” e “Summary”, creare un indicatore booleano che sia:
- True, se i valori della colonna “Summary” nelle ultime 3 ore considerando le righe i-esima, (i-1)-esima e (i-2)-esima sono uguali
- False, altrimenti

In [25]:
# Solution Leon
df['same summary in the last 3 days'] =None
Summary_tail_2=(df['Summary'].shift(-2))
Summary_tail_2=Summary_tail_2.dropna().reset_index(drop=True)
Summary_tail_2=Summary_tail_2.tail(3)
Summary_tail_3=(df['Summary'].shift(-3))
Summary_tail_3=Summary_tail_3.dropna().reset_index(drop=True)
Summary_tail_3=Summary_tail_3.tail(3)
Summary_tail_2 = pd.DataFrame(Summary_tail_2)
Summary_tail_3 = pd.DataFrame(Summary_tail_3)
#df['same summary in the last 3 days'] = [True if (Summary_tail_2[i].equals(Summary_tail_3[i])) else False for i in (df['Formatted Date'].index)] 

In [26]:
# Solution Edo
t = df[['Formatted Date', 'Summary']].copy()
t['Summary (t-1)'] = t['Summary'].shift(1)
t['Summary (t-2)'] = t['Summary'].shift(2)
t['edo'] = t.apply(lambda row: True if row['Summary'] == row['Summary (t-1)'] == row['Summary (t-2)'] else False, axis=1)
t['edo2'] = t.apply(lambda row: True if len(set([row['Summary'], row['Summary (t-1)'], row['Summary (t-2)']])) == 1 else False, axis=1)

#### Assignment 7

Feature “Humidity avg last 2 hours”: a partire dalle colonne “Formatted Date” e “Humidity”, calcolare il valore medio della colonna “Humidity” considerando la riga i-esima e (i-1)-esima, e riportare questo valore nella riga i-esima.

In [27]:
# Solution Leon
df['Humidity avg last 2 days'] =df['Humidity'].shift(-1)
df['Humidity avg last 2 days']= df['Humidity avg last 2 days'].dropna().reset_index(drop=True)
df['Humidity avg last 2 days'] =[ (stat.mean([df['Humidity'][i] , df['Humidity avg last 2 days'][i]])) for i in (df['Formatted Date'].index)]

In [28]:
# Solution Edo
t = df[['Formatted Date', 'Humidity']].copy()
t['Humidity (t-1)'] = t['Humidity'].shift(1)
t['edo'] = t[['Humidity', 'Humidity (t-1)']].mean(axis=1)

#### Assignment 8

Feature “Differenza in Ore”: a partire dalla colonna “Formatted Date”, creare una variabile numerica intera che indica la differenza (misurata in ore) tra due righe consecutive a livello temporale

In [29]:
# Solution Leon
#df['Differenza in Ore'] =df['Formatted Date'].shift(-1)
#df['Differenza in Ore']= df['Formatted Date'].dropna().reset_index(drop=True)
#df['Differenza in Ore']=df['Differenza in Ore'].dt.time
#df['Differenza in Ore']
#df['Formatted Date'] = pd.to_datetime(df['Formatted Date'], utc=True)
#df['Formatted Date']=df['Formatted Date'].dt.time 
#df['Formatted Date']
#df['Differenza in Ore'] =[ (df['Differenza in Ore'][i] -df['Formatted Date'][i])  for i in (df['Formatted Date'].index)]

In [30]:
# Solution Edo
t = df[['Formatted Date']].copy()
t['Formatted Date (t-1)'] = t['Formatted Date'].shift(1)
t['edo'] = ((t['Formatted Date'] - t['Formatted Date (t-1)']).dt.seconds)/(60*60)

In [31]:
t

,Formatted Date,Formatted Date (t-1),edo
0,2009-01-04 16:00:00+00:00,NaT,NaN
1,2009-01-04 17:00:00+00:00,2009-01-04 16:00:00+00:00,1.0
2,2009-01-04 18:00:00+00:00,2009-01-04 17:00:00+00:00,1.0
3,2009-01-04 19:00:00+00:00,2009-01-04 18:00:00+00:00,1.0
4,2009-01-04 20:00:00+00:00,2009-01-04 19:00:00+00:00,1.0
...,...,...,...
95905,2009-12-28 01:00:00+00:00,2009-12-28 00:00:00+00:00,1.0
95906,2009-12-28 02:00:00+00:00,2009-12-28 01:00:00+00:00,1.0
95907,2009-12-28 03:00:00+00:00,2009-12-28 02:00:00+00:00,1.0
95908,2010-01-01 19:00:00+00:00,2009-12-28 03:00:00+00:00,16.0
